# Your info

Full name: Ghazal ZamaniNejad

Student ID: 97522166

Notice: **You can add new cells for each part.**

# Q1

In [15]:
#write your code here
import numpy as np
from numpy import random
import math
import random

class TSP:
  def __init__ (self, cities_count, dists, population, generation):
    self.distances = dists
    self.count = cities_count
    self.population_size = population
    self.generation_size = generation

  def Generate(self):
    arr = np.arange(0, self.count)
    random.shuffle(arr)
    sol = np.append(arr, arr[0])
    return sol

  def Fitness(self, sol):
    total = [self.distances[sol[i], sol[i+1]] for i in range(self.count)]
    # -1 means there's no way, so the solution is not acceptable
    if -1 in total:
      return math.inf
    return sum(total)

  def Reproduction(self, fit, k):
    # use Elitist and choose the parents with better fitness (lower fitntess)
    idx = np.argpartition(fit, k)
    # return a list of parents' index
    return idx[:k]

  def CrossOver(self, p1, p2):
    # use order1 crossover
    child = []
    (i, j) = random.sample(range(self.count), 2)
    a = min(i, j)
    b = max(i, j)

    for i in range(a, b):
      child.append(p1[i])
    c = [item for item in p2 if item not in child]
    child.extend(c)
    
    if (len(child) != self.count + 1):
      child.append(child[0])
    else: child[len(child) -1 ] = child[0]
    
    return child

  def Mutation(self, m):
    (i, j) = random.sample(range(self.count), 2)
    
    (m[i], m[j]) = (m[j], m[i])
    m[len(m) -1] = m[0]
    return m

  def GA(self):
    # initialize population 
    # and calculate fitness of each
    pop = []
    fitness = []

    for i in range(self.population_size):
      sol = self.Generate()
      pop.append(sol)
      fitness.append(self.Fitness(sol))

    # do the loop for "generation" times
    for g in range(self.generation_size):
      # do reproduction and choose parents
      k = 4 # number of parents
      indexes = self.Reproduction(fitness, k)
      parents = []
      for index in indexes:
        parents.append(pop[index])

      children = []
      # do crossover
      for i in range(len(parents)):
        for j in range(len(parents)):
          if j == i:
            continue
          child = self.CrossOver(parents[i], parents[j])
          children.append(child)
      
      pop = []
      fitness = []
      for p in parents:
        pop.append(p)
        fitness.append(self.Fitness(p))

      # do mutation for each new child
      for child in children:
        child = self.Mutation(child)
        pop.append(child)
        fitness.append(self.Fitness(child))

      if g % 40 == 0:
        best = np.argmin(fitness) 
        answer = [x+1 for x in pop[best]]
        print("best solution, iteration", g, ":", answer)
        print("length", fitness[best])

    best = np.argmin(fitness) 
    answer = [x+1 for x in pop[best]]
    print("best solution is:", answer)
    print("length:", fitness[best])

dists = np.array([[0, 12, 10, -1, -1, -1, 12],
                  [12, 0, 8, 12, -1, -1, -1],
                  [10, 8, 0, 11, 3, -1, 9],
                  [-1, 12, 11, 0, 11, 10, -1],
                  [-1, -1, 3, 11, 0, 6, 7],
                  [-1, -1, -1, 10, 6, 0, 9],
                  [12, -1, 9, -1, 7, 9, 0]
                  ])

tsp = TSP(7, dists, 10, 200)
tsp.GA()


best solution, iteration 0 : [7, 5, 3, 1, 2, 4, 6, 7]
length 63
best solution, iteration 40 : [2, 1, 3, 5, 7, 6, 4, 2]
length 63
best solution, iteration 80 : [2, 1, 3, 5, 7, 6, 4, 2]
length 63
best solution, iteration 120 : [2, 1, 3, 5, 7, 6, 4, 2]
length 63
best solution, iteration 160 : [3, 5, 7, 6, 4, 2, 1, 3]
length 63
best solution is: [4, 2, 1, 3, 5, 7, 6, 4]
length: 63


# Q2

In [16]:
#write your code here
import numpy as np
from numpy import random
import math
import random

class Equation:
  def __init__ (self, ch, population, generation):
    self.chromosome = ch
    self.population_size = population
    self.generation_size = generation

  def Value(self, x):
    return 9 * x**5 -194.7 * x**4 + 1680.1 * x**3 - 7227.94 * x**2 + 15501.2 * x - 13257.2

  def Generate(self):
    sol = random.choices(range(10), k=6)
    # sol[0] is even ? positive : negative
    return sol

  def ConvertListToNum(self, x):
    # i = int(''.join(map(str, x[1:2])))
    i = int(x[1])
    f = float(''.join(map(str, x[2:]))) * 0.001
    num = i + f
    if (x[0] % 2 == 1):
      num *= -1
    return num

  def Fitness(self, sol):
    x = self.ConvertListToNum(sol)
    return abs(self.Value(x))

  def Reproduction(self, fit, k):
    # use Elitist and choose the parents with better fitness (lower fitntess)
    idx = np.argpartition(fit, k)
    return idx[:k]

  def CrossOver(self, p1, p2):
    # use two-point crossover
    ch1 = np.copy(p1)
    ch2 = np.copy(p2)
    (i, j) = random.sample(range(self.chromosome), 2)
    
    if (i > j):
      for k in range(i, self.chromosome):
        (ch1[k], ch2[k]) = (ch2[k], ch1[k])

      for k in range(j+1):
        (ch1[k], ch2[k]) = (ch2[k], ch1[k])

    else:
      for k in range(i, j+1):
        (ch1[k], ch2[k]) = (ch2[k], ch1[k])

    return (ch1, ch2)

  def Mutation(self, m):
    (i, j) = random.sample(range(self.chromosome), 2)
    (m[i], m[j]) = (m[j], m[i])
    return m

  def GA(self):
    # initialize population 
    # and calculate fitness of each
    pop = []
    fitness = []

    for i in range(self.population_size):
      sol = self.Generate()
      pop.append(sol)
      fitness.append(self.Fitness(sol))

    # do the loop for "generation" times
    for g in range(self.generation_size):
      # do reproduction and choose parents
      k = 4
      indexes = self.Reproduction(fitness, k)
      parents = []
      for index in indexes:
        parents.append(pop[index])

      children = []
      # do crossover
      for i in range(len(parents)-1):
        for j in range(i+1, len(parents)):
          ch1, ch2 = self.CrossOver(parents[i], parents[j])
          children.append(ch1)
          children.append(ch2)
      
      pop = []
      fitness = []
      for p in parents:
        pop.append(p)
        fitness.append(self.Fitness(p))

      # do mutation for each new child
      for child in children:
        child = self.Mutation(child)
        pop.append(child)
        fitness.append(self.Fitness(child))

    best = np.argmin(fitness) 
    print("best solution is:", self.ConvertListToNum(pop[best]))
    print("f(solution) = ", fitness[best])

# 1 bit --> sign
# 1 + 4 bits --> float number
# -9.9999 to +9.9999
equ = Equation(6, 500, 2000)
equ.GA()

best solution is: 4.884
f(solution) =  0.00010777905481518246


# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='red'>**D5372R**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
   

  according to the question number.
  - There are 2 assignments in Gradescope: 

    You should upload your **codes** and your **notebook** in `HW4` section and your final report for all of the questions as a **single pdf** file in `HW4 - Report`. Autograder will automatically check for:
    - `CI992_HW4.ipynb`
    - `Q1 Q1.py`
    - `Q2 Q2.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.